Feature Engineering com LLM (Produtos Industriais)

In [13]:
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer

In [14]:
PRODUCTS_PATH = "../data/trusted/products_trusted.parquet"
CUSTOMERS_PATH = "../data/trusted/customers_trusted.parquet"
SALES_PATH = '../data/trusted/sales_trusted.parquet'

products = pd.read_parquet(PRODUCTS_PATH)
customers = pd.read_parquet(CUSTOMERS_PATH)
sales = pd.read_parquet(SALES_PATH)

print(products.shape, customers.shape, sales.shape)

(10000, 17) (5000, 14) (120000, 14)


In [15]:
# Já temos technical_description e technical_features do notebook 02
# Vamos apenas confirmar que existem
print("✓ Campos disponíveis:")
print(f"  - technical_description: {type(products['technical_description'].iloc[0])}")
print(f"  - technical_features: {type(products['technical_features'].iloc[0])}")
print(f"  - llm_product_description: {type(products['llm_product_description'].iloc[0])}")

✓ Campos disponíveis:
  - technical_description: <class 'str'>
  - technical_features: <class 'numpy.ndarray'>
  - llm_product_description: <class 'str'>


In [16]:
# problem_keywords = {
#     "Vibração": ["vibration", "stability", "balance"],
#     "Desgaste": ["wear", "durability", "long life"],
#     "Superaquecimento": ["heat", "temperature", "cooling"],
#     "Corrosão": ["corrosion", "stainless", "humidity"],
#     "Contaminação": ["sealed", "hygiene", "food"]
# }

# def infer_supported_problems(text):
#     text_lower = text.lower()
#     supported = []
    
#     for problem, keywords in problem_keywords.items():
#         if any(k in text_lower for k in keywords):
#             supported.append(problem)
    
#     return supported if supported else ["Uso Geral"]

# products['supported_problems'] = products['llm_product_description'].apply(
#     infer_supported_problems
# )

problem_keywords = {
    "Vibração": ["vibration", "stability", "balance", "oscilação", "vibr"],
    "Desgaste": ["wear", "durability", "long life", "desgaste", "erosão"],
    "Superaquecimento": [
        "overheating", "superaquec", 
        "alta temperatura", "temperatura elevada",
        "heat build-up", "excessive heat"
    ],
    "Corrosão": ["corrosion", "stainless", "humidity", "corros", "oxidaç"],
    "Contaminação": ["sealed", "hygiene", "food", "contam", "isolad"],
}

def infer_supported_problems(text):
    if not isinstance(text, str) or not text:
        return ["Uso Geral"]
    
    text_lower = text.lower()
    supported = []
    
    for problem, keywords in problem_keywords.items():
        if any(k in text_lower for k in keywords):
            supported.append(problem)
    
    return supported if supported else ["Uso Geral"]


# Aplicar com validação
products['supported_problems'] = products['llm_product_description'].apply(
    infer_supported_problems
)

# Verificar resultado
print("Distribuição de problemas encontrados:")
print(products['supported_problems'].apply(len).value_counts())
print("\nExemplos:")
print(products['supported_problems'].head(10))


Distribuição de problemas encontrados:
supported_problems
1    10000
Name: count, dtype: int64

Exemplos:
0            [Vibração]
1        [Contaminação]
2            [Vibração]
3            [Vibração]
4            [Vibração]
5            [Vibração]
6    [Superaquecimento]
7            [Desgaste]
8        [Contaminação]
9    [Superaquecimento]
Name: supported_problems, dtype: object


In [17]:

# Garantir que expected_problems seja lista antes de passar pro MLB
print("\n Verificando tipo de expected_problems...")
first_val = customers['expected_problems'].iloc[0]
print(f"Tipo: {type(first_val)}, Valor: {first_val}")

# Se for string, converter para lista
if isinstance(first_val, str):
    print("   ⚠️  Convertendo strings para listas...")
    customers['expected_problems'] = customers['expected_problems'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x
    )
    print("Conversão concluída")

# Agora sim, fazer o encoding
mlb_customers = MultiLabelBinarizer()
customer_problem_features = mlb_customers.fit_transform(customers['expected_problems'])

customer_problem_df = pd.DataFrame(
    customer_problem_features,
    columns=[f"problem_{c.lower().replace(' ', '_').replace('ã', 'a').replace('ç', 'c').replace('õ', 'o')}" for c in mlb_customers.classes_]
)

customers_ml = pd.concat(
    [customers.reset_index(drop=True), customer_problem_df],
    axis=1
)

print(f"\n Features binárias criadas: {list(customer_problem_df.columns)}")
mlb = MultiLabelBinarizer()

problem_features = mlb.fit_transform(products['supported_problems'])

problem_features_df = pd.DataFrame(
    problem_features,
    columns=[f"problem_{c}" for c in mlb.classes_]
)

products_ml = pd.concat(
    [products.reset_index(drop=True), problem_features_df],
    axis=1
)


 Verificando tipo de expected_problems...
Tipo: <class 'numpy.ndarray'>, Valor: ['Contaminação' 'Superaquecimento']

 Features binárias criadas: ['problem_contaminacao', 'problem_desgaste', 'problem_superaquecimento', 'problem_vibracao']


In [18]:
FEATURES_PATH = "../data/refined/"

products_ml.to_parquet(f"{FEATURES_PATH}/products_features.parquet", index=False)
customers_ml.to_parquet(f"{FEATURES_PATH}/customers_features.parquet", index=False)

print("Features geradas e salvas com sucesso")

Features geradas e salvas com sucesso


In [19]:
# ===== SALVAR SALES NA REFINED =====

# Conversões básicas
sales['sale_date'] = pd.to_datetime(sales['sale_date'])
sales['last_updated'] = pd.to_datetime(sales['last_updated'])

# Salvar
sales.to_parquet('../data/refined/sales_refined.parquet', index=False)

print("✅ sales_refined.parquet criado em ../data/refined/")


✅ sales_refined.parquet criado em ../data/refined/


In [20]:
# Selecionar colunas problem_* e converter tudo para numérico
problem_df = products_ml.filter(like="problem_").apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
print(problem_df.sum().sort_values(ascending=False))

problem_Contaminação        2559
problem_Vibração            2528
problem_Superaquecimento    2466
problem_Desgaste            2447
problem_type                   0
dtype: int64


In [21]:
customers_ml.filter(like="problem_").sum().sort_values(ascending=False)

problem_vibracao            3126
problem_superaquecimento    2515
problem_desgaste            2485
problem_contaminacao        1874
dtype: int64